In [1]:
from pathlib import Path

root = Path('./road_sign_datasets')
Path('./road_sign_datasets/train').mkdir(parents=True, exist_ok=True)
Path('./road_sign_datasets/val').mkdir(parents=True, exist_ok=True)

In [2]:
for path1 in ('images', 'labels'):
    for path2 in ('train', 'val', 'test'):
        new_path = root / path1 / path2
        new_path.mkdir(parents=True, exist_ok=True)

In [15]:
%cd /content/road_sign_datasets/train

/content/road_sign_datasets/train


In [17]:
!unzip -q train.zip

In [18]:
%cd /content/road_sign_datasets/val

/content/road_sign_datasets/val


In [19]:
!unzip -q val.zip

In [24]:
%cd /content/road_sign_datasets

/content/road_sign_datasets


In [25]:
!git clone https://github.com/ssaru/convert2Yolo.git

fatal: destination path 'convert2Yolo' already exists and is not an empty directory.


In [26]:
%cd convert2Yolo/

/content/road_sign_datasets/convert2Yolo


In [10]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 16.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [27]:
!python example.py \
    --datasets VOC \
    --img_path /content/road_sign_datasets/train/images \
    --label /content/road_sign_datasets/train/labels \
    --convert_output_path /content/road_sign_datasets/labels/train \
    --img_type '.png' \
    --manifest_path /content/road_sign_datasets \
    --cls_list_file /content/road_sign_datasets/voc_road.names


VOC Parsing:   |████████████████████████████████████████| 100.0% (701/701)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (701/701)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (701/701)  Complete



In [28]:
# val

!python example.py \
    --datasets VOC \
    --img_path /content/road_sign_datasets/val/images \
    --label /content/road_sign_datasets/val/labels \
    --convert_output_path /content/road_sign_datasets/labels/val \
    --img_type '.png' \
    --manifest_path /content/road_sign_datasets \
    --cls_list_file /content/road_sign_datasets/voc_road.names


VOC Parsing:   |████████████████████████████████████████| 100.0% (176/176)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (176/176)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (176/176)  Complete



In [29]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.6 MB/s eta 0:00:00


In [30]:
import os, shutil

# val 이미지 이름 기준 (확장자 제외)
val_img_dir = '/content/road_sign_datasets/images/val'

train_label_dir = '/content/road_sign_datasets/labels/train'
val_label_dir = '/content/road_sign_datasets/labels/val'

for file in os.listdir(val_img_dir):
    img_id = os.path.splitext(file)[0]   # road701
    src = f'{train_label_dir}/{img_id}.txt'
    dst = f'{val_label_dir}/{img_id}.txt'

    if os.path.exists(src):
        shutil.move(src, dst)

In [31]:
base = '/content/road_sign_datasets'

label_dir = f'{base}/labels/train'
src_img_dir = f'{base}/train/images'   # 기존 이미지 위치
dst_img_dir = f'{base}/images/train'   # YOLO용 위치

for label in os.listdir(label_dir):
    img_name = label.replace('.txt', '.png')  # png면 png, jpg면 jpg
    src = f'{src_img_dir}/{img_name}'
    dst = f'{dst_img_dir}/{img_name}'

    if os.path.exists(src):
        shutil.move(src, dst)

In [32]:
label_dir = f'{base}/labels/val'
src_img_dir = f'{base}/val/images'
dst_img_dir = f'{base}/images/val'

for label in os.listdir(label_dir):
    img_name = label.replace('.txt', '.png')
    src = f'{src_img_dir}/{img_name}'
    dst = f'{dst_img_dir}/{img_name}'

    if os.path.exists(src):
        shutil.move(src, dst)

In [33]:
LABEL_DIR = '/content/road_sign_datasets/labels/train'

removed_files = 0
fixed_files = 0
total_boxes = 0

for file in os.listdir(LABEL_DIR):
    if not file.endswith('.txt'):
        continue

    path = os.path.join(LABEL_DIR, file)

    with open(path, 'r') as f:
        lines = f.readlines()

    clean_lines = []

    for line in lines:
        line = line.strip()

        # None 포함 라인 제거
        if 'None' in line or line == '':
            continue

        parts = line.split()

        # YOLO 형식 체크 (5개 값)
        if len(parts) != 5:
            continue

        try:
            cls, x, y, w, h = parts
            cls = int(cls)
            x, y, w, h = map(float, (x, y, w, h))
        except:
            continue

        # 값 범위 체크
        if not (0 <= x <= 1 and 0 <= y <= 1):
            continue
        if not (0 < w <= 1 and 0 < h <= 1):
            continue

        clean_lines.append(f"{cls} {x} {y} {w} {h}\n")

    # 결과 처리
    if len(clean_lines) == 0:
        os.remove(path)
        removed_files += 1
    else:
        with open(path, 'w') as f:
            f.writelines(clean_lines)
        fixed_files += 1
        total_boxes += len(clean_lines)

print(f"정상 라벨 파일 수 : {fixed_files}")
print(f"삭제된 라벨 파일 수 : {removed_files}")
print(f"총 bounding box 수 : {total_boxes}")

정상 라벨 파일 수 : 648
삭제된 라벨 파일 수 : 53
총 bounding box 수 : 812


In [34]:
label_dir = '/content/road_sign_datasets/labels/val'

valid_files = 0
removed_files = 0
total_boxes = 0

for file in os.listdir(label_dir):
    path = os.path.join(label_dir, file)

    with open(path, 'r') as f:
        lines = f.readlines()

    clean_lines = []
    for line in lines:
        parts = line.strip().split()

        # YOLO 라벨은 반드시 5개 숫자여야 함
        if len(parts) != 5:
            continue

        try:
            nums = list(map(float, parts))
            clean_lines.append(line)
            total_boxes += 1
        except:
            continue

    if len(clean_lines) == 0:
        os.remove(path)
        removed_files += 1
    else:
        with open(path, 'w') as f:
            f.writelines(clean_lines)
        valid_files += 1

print(f"정상 라벨 파일 수 : {valid_files}")
print(f"삭제된 라벨 파일 수 : {removed_files}")
print(f"총 bounding box 수 : {total_boxes}")

정상 라벨 파일 수 : 176
삭제된 라벨 파일 수 : 0
총 bounding box 수 : 262


In [35]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [36]:
model = YOLO('yolov8s.pt')

In [39]:
results = model.train(
    data='/content/road_sign_datasets/data.yaml',
    epochs=10,
    batch=32,
    imgsz=640,
    device=0,
    workers=2,
    name='custom_s'
)

Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/road_sign_datasets/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=custom_s3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=Tr

In [40]:
# 학습된 모델 로드
model = YOLO("runs/detect/custom_s3/weights/best.pt")

# 검증 수행
results = model.predict(
    source="/content/road_sign_datasets/images/val",
    conf=0.25,
    save=True
)

print(results)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
       [[102, 115, 110],
        [106, 116, 114],
        [108, 119, 116],
        ...,
        [ 78,  99,  96],
        [ 92, 113, 110],
        [ 92, 115, 111]],

       [[101, 114, 109],
        [102, 114, 110],
        [104, 116, 112],
        ...,
        [ 75,  99,  93],
        [ 84, 109, 103],
        [ 88, 114, 108]]], dtype=uint8)
orig_shape: (400, 300)
path: '/content/road_sign_datasets/images/val/road793.png'
probs: None
save_dir: '/content/road_sign_datasets/convert2Yolo/runs/detect/predict'
speed: {'preprocess': 1.7825789998369146, 'inference': 7.080318999896917, 'postprocess': 1.2587890000759216}, ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'traficlight', 1: 'stop', 2: 'speedlimit', 3: 'crosswalk'}
obb: None
orig_img: array([[[215, 190, 125],
        [217, 190, 127],
        [217, 190, 127],
        ...,
        [217, 190, 127],
      